We import `netgen.gui` to open the netgen GUI for displaying the ferromagnet. 
We then define the geometry, and draw the box. 
The refinement of the mesh is controlled by `H_MAX` in `ngmesh`.
The maximum time for the simulation is given by `T_MAX`, and has time step `K`.
Use `THETA` to determine "how implicit the tangent plane scheme solver is". Closer to 1/2 is usually better, but must be above 1/2 for unconditional convergence.

In [ ]:
from netgen.csg import *
from ngsolve import *
from ngsolve.utils import (
    Grad
)  # If I don't import these explicitly, VSCode reads them as missing.
import netgen.gui  # this opens up the netgen ui
import Magnetisation_Functions as magfunc
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
import Elastic_Functions as elfunc
import General_Functions as genfunc
import sys
import time
np.set_printoptions(threshold=sys.maxsize)


# MATERIAL PARAMETERS
M_s = 1.5e6
gyromagnetic_ratio = 1.761e11
exchange_length = 3e-9
lambda_m: float = 30e-6  # saturation magnetostrain parameter
mu_0 = 1.25663706e-6
density = 7900
KAPPA: float = genfunc.calculate_KAPPA(density, exchange_length, gyromagnetic_ratio, mu_0)  # Determines the relative strength of the elastic vs. magnetic parts.
stress_fac = genfunc.stress_density_factor(KAPPA=KAPPA, mu_0=mu_0, M_s=M_s)
mu, lam = 54e9/stress_fac, 172e9/stress_fac
ALPHA: float = 1  # Dissipative constant in the LLG equation.

# SIMULATION PARAMETERS
T_MAX_DIM: float = 1e-9  # The maximum time for the simulation in seconds
THETA: float = 0.50000005  # Should be strictly above 1/2 for unconditional stability, but close to 1/2 to reduce (artificial) numerical dissipation
time_step_dim = 5e-13 # seconds
K: float = time_step_dim*genfunc.nondimensional_time(gyromagnetic_ratio, mu_0, M_s=M_s)  # TIME STEP
T_MAX: float = T_MAX_DIM*genfunc.nondimensional_time(gyromagnetic_ratio, mu_0, M_s=M_s)
H_MAX: float = 0.7  # Determines how fine the mesh should be.

print(f"µ = {mu}, λ = {lam}, κ = {KAPPA}")
print(f"K = {K}, T_MAX = {T_MAX}")

$\boldsymbol\sigma_{ij} = 2\mu \boldsymbol\varepsilon_{ij}^{\text{el}} + \lambda \delta_{ij}\boldsymbol\varepsilon_{kk}^{\text{el}}$ Hooke's Law

$\mathbb{Z}_{ijkl} \boldsymbol E_{kl} = \frac{3\lambda_{s}}{2}\left(\boldsymbol E_{ij} - \frac{1}{3}\delta_{ij}\boldsymbol E_{kk}\right)$  Isotropic, isochoric magnetostrain for some symmetric input

Here we make a cube with labelled faces.

In [ ]:
def MakeGeometry():  # this makes a box, with labelled faces
    geometry = CSGeometry()
    left = Plane(Pnt(0, 0, 0), Vec(-1, 0, 0)).bc("left")
    right = Plane(Pnt(20, 20, 20), Vec(1, 0, 0)).bc("right")
    front = Plane(Pnt(0, 0, 0), Vec(0, -1, 0)).bc("front")
    back = Plane(Pnt(6, 6, 6), Vec(0, 1, 0)).bc("back")
    bot = Plane(Pnt(0, 0, 0), Vec(0, 0, -1)).bc("bot")
    top = Plane(Pnt(6, 6, 6), Vec(0, 0, 1)).bc("top")

    cube = left * right * front * back * bot * top
    geometry.Add(cube)
    # cube = OrthoBrick(Pnt(0,0,0), Pnt(1,1,1))
    return geometry


ngmesh = MakeGeometry().GenerateMesh(maxh=H_MAX)
# ngmesh.Save("cube.vol")
mesh = Mesh(ngmesh)
Draw(mesh)
genfunc.MaximumMeshSize(mesh)

We make two FE spaces, one for magnetisation `fes_mag`, and one for displacement/velocity `fes_disp`. (and a legacy FE space for matrices `fes_eps_m`)

The displacement has a `dirichlet` condition, in this case on the `left` face.

We then introduce `f_body`, which is a body force density. We use simply gravity here. You can also set a traction on specific faces with `g_surface`.

In [ ]:
fes_mag = VectorH1(
    mesh, order=1
)  # the finite element space for the magnetisation m_h^i
fes_disp = VectorH1(
    mesh, order=1, dirichlet="left"
)  # the finite element space for the displacement u_h^i
fes_scalar = H1(mesh, order=1)
print(f"mag_ndof={fes_mag.ndof}, disp_ndof={fes_disp.ndof},\n, dispfree_ndof={fes_disp.FreeDofs()}")
mag_gfu = GridFunction(fes_mag)
disp_gfu = GridFunction(fes_disp)
prev_disp_gfu = GridFunction(fes_disp) #  used to store the previous displacement
zeeman_factor = Parameter(0)
f_zeeman = CoefficientFunction((0, 0, zeeman_factor))
# body force and traction force
body_factor = Parameter(genfunc.force_density_grav(grav_accel=-9.81, density=density, exchange_length=exchange_length, KAPPA=KAPPA, mu_0=mu_0, M_s=M_s))
f_body = CoefficientFunction((0.0, 0.0, body_factor))
surface_factor = Parameter(1e-6)
g_surface = CoefficientFunction([(0,surface_factor,0) if bc=="right" else (0,0,0) for bc in mesh.GetBoundaries()])
Draw(g_surface, mesh, "gsruface")
Draw(f_body, mesh, "fbody")
print(body_factor)

Here we give initial conditions for the magnetisation and displacement.

The magnetisation can either be given fully random, or uniform magnetisation. If you choose uniform, you must pick the direction. It will be automatically normalised.

The displacement can also be given a fully random or uniform. For the displacement, you can choose the largest magnitude for the random displacement.

In [ ]:
# Initial conditions
mag_gfu = magfunc.give_uniform_magnetisation(mag_gfu, (1,0,0))
#disp_gfu = elfunc.give_random_displacement(disp_gfu)
disp_gfu = elfunc.give_uniform_displacement(disp_gfu, (0.0, 0.0, 0.0))  # initial displacement
velocity_gfu = GridFunction(fes_disp)  # velocity
# make initial conditions compatible with Dirichlet boundary conditions
disp_gfu.Set(CoefficientFunction((0,0,0)), BND)
velocity_gfu.Set(CoefficientFunction((0,0,0)), BND)
velocity_gfu = elfunc.give_uniform_displacement(velocity_gfu, (0.0, 0.0, 0.0))  # An initial velocity. Should only be used once in iteration.
Draw(mag_gfu, mesh, "magnetisation")
Draw(disp_gfu, mesh, "displacement")
proj_mag = magfunc.nodal_projection(mag_gfu, fes_mag)
strain_m = magfunc.build_strain_m(proj_mag, lambda_m)
box_vol = Integrate(CF(1.0), mesh)
current_integral = magfunc.constraint_error(fes_scalar, mag_gfu, mesh)
current_nodal_norm = magfunc.nodal_norm(mag_gfu)
print(f"_/‾|m|^2dx = {current_integral}, nodal max = {current_nodal_norm} (should both be 1)")

In [ ]:
len(mag_gfu.vec)

These are lists of values to be used in plotting later, to demonstrate that the `energy` decreases over time. The `integral_list` is a list of true integrals $\int_{\Omega}|\mathbf{m}_{h}^{i}|^2 \mathrm{d}x / \int_{\Omega}\mathrm{d}x$, and the `nodal_sum_list` is a list of the sum of nodal magnitudes $\max_{z_{k}\in\mathcal{N}_{h}}|\mathbf{m}_{h}^{i}(z_{k})|^2$. The `nodal_sum_list` should be strictly increasing if the tangent plane scheme is functioning. 

In [ ]:
energy_list = np.array([magfunc.magnetic_energy(mag_gfu, mesh, f_zeeman) + elfunc.elastic_energy(mesh, disp_gfu, strain_m, f_body, g_surface, KAPPA, mu, lam)])
kin_energy_list = np.array([elfunc.initial_kinetic_energy(mesh, velocity_gfu, KAPPA)])
integral_list = np.array([magfunc.constraint_error(fes_scalar, mag_gfu, mesh)])
nodal_sum_list= np.array([magfunc.nodal_norm(mag_gfu)])
x_avg, y_avg, z_avg = magfunc.component_integrator(mag_gfu, mesh, box_vol)
x_average_list, y_average_list, z_average_list = np.array([x_avg]), np.array([y_avg]), np.array([z_avg])


In [ ]:
genfunc.export_to_vtk_file(disp_gfu, mag_gfu, mesh, export=True, index=0)

This is the first run of the simulation, which needs to be treated differently as we have a prescribed initial velocity $\dot{\mathbf{u}}^{0}$. First compute the update $\mathbf{v}_{h}^{i}$ to get $\mathbf{m}_{h}^{i+1} = \mathbf{m}_{h}^{i} + \mathbf{v}_{h}^{i}$ using the tangent plane scheme (with GMRES). Then using the conservation of momentum equation calculate $\mathbf{u}_{h}^{i+1}$ using the internal solver of NGSolve.

In [ ]:
num_steps = genfunc.ceiling_division(T_MAX, K)  # This is ceiling division, using upside-down floor division.
#first run
M_mag_fixed, L_mag_fixed = magfunc.build_fixed_mag(fes_mag, ALPHA, THETA, K)  # static portion of the magnetisation linear system
v0 = np.zeros(fes_mag.ndof)
mag_gfu, v0 = magfunc.update_magnetisation(fes_mag, mag_gfu, ALPHA, THETA, K, KAPPA, disp_gfu, mu, lam, lambda_m, f_zeeman, M_mag_fixed, L_mag_fixed, v0)

proj_mag = magfunc.nodal_projection(mag_gfu, fes_mag)
strain_m = magfunc.build_strain_m(proj_mag, lambda_m)
new_disp = elfunc.FIRST_RUN_update_displacement(fes_disp, disp_gfu, velocity_gfu, strain_m, f_body, g_surface, K, mu, lam)
prev_disp_gfu.vec.data = disp_gfu.vec.data
disp_gfu.vec.data = new_disp.vec.data

current_integral = magfunc.constraint_error(fes_scalar, mag_gfu, mesh)
current_nodal_norm = magfunc.nodal_norm(mag_gfu)
current_energy = magfunc.magnetic_energy(mag_gfu, mesh, f_zeeman) + elfunc.elastic_energy(mesh, disp_gfu, strain_m, f_body, g_surface, KAPPA, mu, lam)
current_kinetic_energy = elfunc.kinetic_energy(mesh, disp_gfu, prev_disp_gfu, KAPPA, K)


integral_list = np.append(integral_list, current_integral)
nodal_sum_list = np.append(nodal_sum_list, current_nodal_norm)
energy_list = np.append(energy_list, current_energy)
kin_energy_list = np.append(kin_energy_list, current_kinetic_energy)
x_avg, y_avg, z_avg = magfunc.component_integrator(mag_gfu, mesh, box_vol)
x_average_list, y_average_list, z_average_list = np.append(x_average_list, x_avg), np.append(y_average_list, y_avg), np.append(z_average_list, z_avg)

In [ ]:
mip = mesh(0.0, 0.0, 0)
print(strain_m(mip))

In [ ]:
# subsequence runs
print(f"num_steps={num_steps}")
for i in range(1, num_steps):
    time_start = time.time()
    mag_gfu, v0 = magfunc.update_magnetisation(fes_mag, mag_gfu, ALPHA, THETA, K, KAPPA, disp_gfu, mu, lam, lambda_m, f_zeeman, M_mag_fixed, L_mag_fixed, v0)
    proj_mag = magfunc.nodal_projection(mag_gfu, fes_mag)
    strain_m = magfunc.build_strain_m(proj_mag, lambda_m)
    new_disp = elfunc.update_displacement(fes_disp, disp_gfu, prev_disp_gfu, strain_m, f_body, g_surface, K, mu, lam)
    prev_disp_gfu.vec.data = disp_gfu.vec.data
    disp_gfu.vec.data = new_disp.vec.data
    #Redraw()
    current_integral = magfunc.constraint_error(fes_scalar, mag_gfu, mesh)
    current_nodal_norm = magfunc.nodal_norm(mag_gfu)
    current_energy = magfunc.magnetic_energy(mag_gfu, mesh, f_zeeman) + elfunc.elastic_energy(mesh, disp_gfu, strain_m, f_body, g_surface, KAPPA, mu, lam)
    current_kinetic_energy = elfunc.kinetic_energy(mesh, disp_gfu, prev_disp_gfu, KAPPA, K)
    print(f"Step {i}:_/‾I_h(|m|^2) - 1 dx = {current_integral}, nodal max = {current_nodal_norm} (should both be 1)\n\
        energy      = {current_energy}")
    print(f"kinetic energy = {current_kinetic_energy}")
    genfunc.export_to_vtk_file(disp_gfu, mag_gfu, mesh, export=True, index=i, save_step=10)
    integral_list = np.append(integral_list, current_integral)
    nodal_sum_list = np.append(nodal_sum_list, current_nodal_norm)
    energy_list = np.append(energy_list, current_energy)
    kin_energy_list = np.append(kin_energy_list, current_kinetic_energy)
    x_avg, y_avg, z_avg = magfunc.component_integrator(mag_gfu, mesh, box_vol)
    x_average_list, y_average_list, z_average_list = np.append(x_average_list, x_avg), np.append(y_average_list, y_avg), np.append(z_average_list, z_avg)
    time_end = time.time()
    print(f"estimated remaining time = {genfunc.est_time_remaining(num_steps, i, time_end-time_start)}")


Now we plot the `energy_list`, `kin_energy_list` and also the `integral_list`+`nodal_sum_list`.

In [ ]:
my_time = np.linspace(0, T_MAX_DIM, num_steps+1)

plt.plot(my_time, energy_list+kin_energy_list, label="Total Energy")
plt.xlabel("Time")
plt.title(f"dt = {time_step_dim}, total = {T_MAX_DIM}, {num_steps} steps")
plt.legend()
plt.savefig("total_energy_graph.pdf")

In [ ]:
my_time = np.linspace(0, T_MAX_DIM, num_steps+1)

plt.plot(my_time, energy_list, label="Pot. Energy")
plt.xlabel("Time")
plt.title(f"dt = {time_step_dim}, total = {T_MAX_DIM}, {num_steps} steps")
plt.legend()
plt.savefig("potential_energy_graph.pdf")


In [ ]:

plt.plot(my_time, kin_energy_list, label="Kin. Energy")
plt.xlabel("Time")
plt.title(f"dt = {time_step_dim}, total = {T_MAX_DIM}, {num_steps} steps")
plt.legend()
plt.savefig("kinetic_graph.pdf")

In [ ]:
plt.plot(my_time, integral_list, label=r"$\int_{\Omega}|\mathcal{I}_{h}(|\mathbf{m}_{h}^{i}|^2)-1| \mathrm{d}x$")
plt.xlabel(r"time")
plt.legend()
plt.savefig("integral_graph.pdf")

In [ ]:
plt.plot(my_time, nodal_sum_list, label=r"$\max |\mathbf{m}(z_k)|^2$")
plt.xlabel(r"time")
plt.legend()
plt.savefig("nodalmax_graph.pdf")

In [ ]:
totalenergy = energy_list+kin_energy_list
np.save("totalenergy", totalenergy)
np.save("potentialenergy", energy_list)
np.save("kinenergy", kin_energy_list)
np.save("integrals", integral_list)
np.save("mytimelist", my_time)
np.save("x_average", x_average_list)
np.save("y_average", y_average_list)
np.save("z_average", z_average_list)

$\int_{\Omega}|\mathcal{I}_{h}(|\mathbf{m}_{h}^{N}|^2)-1| \mathrm{d}x$

In [ ]:
print(f"Constraint error is {magfunc.constraint_error(fes_scalar, mag_gfu, mesh)}")

In [ ]:
plt.plot(my_time, x_average_list, label=r"$\langle m_{x} \rangle$")
plt.xlabel(r"time (s)")
plt.legend()
plt.savefig("x_average_graph.pdf")

In [ ]:
plt.plot(my_time, y_average_list, label=r"$\langle m_{y} \rangle$")
plt.xlabel(r"time (s)")
plt.legend()
plt.savefig("y_average_graph.pdf")

In [ ]:
plt.plot(my_time, z_average_list, label=r"$\langle m_{z} \rangle$")
plt.xlabel(r"time (s)")
plt.legend()
plt.savefig("z_average_graph.pdf")